In [1]:
import chromadb
import torch
import re
import os

from datasets import load_dataset

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from chromadb import PersistentClient



from transformers import (
    AutoModel, AutoTokenizer,AutoModelForCausalLM,TrainingArguments, DataCollatorForLanguageModeling, Trainer
)

from week5.tools.get_embedding import get_embedding

from datasets import load_dataset

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [2]:
qwen_model_path = '/home/wby/projects/model/Qwen-7B-Chat'
db_path = '/home/wby/projects/week5/chroma_db/chroma_recipe_db'
model_path = '/home/wby/projects/model/all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def load_qwen_tokenizer(qwen_model_path):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            qwen_model_path,
            trust_remote_code=True,
        )
        qwen_official_chat_template = """
        {% for message in messages %}
            {% if message['role'] == 'user' %}
                <|im_start|>user
                {{ message['content'] }}
                <|im_end|>
            {% elif message['role'] == 'assistant' %}
                <|im_start|>assistant
                {{ message['content'] }}
                <|im_end|>
            {% endif %}
        {% endfor %}
        {% if add_generation_prompt %}
            <|im_start|>assistant
        {% endif %}
        """
        tokenizer.chat_template = qwen_official_chat_template
        # 配置pad_token（和之前一致）
        if tokenizer.eos_token is None:
            tokenizer.eos_token = "<|endoftext|>"
            tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        print("分词器加载成功！")
        return tokenizer
    except Exception as e:
        print(f"分词器加载失败：{e}")
        return None

qwen_tokenizer = load_qwen_tokenizer(qwen_model_path)

分词器加载成功！


In [4]:
from transformers import BitsAndBytesConfig

def load_qwen_model(qwen_model_path):
    try:
        print("开始加载模型，模型较大，耐心等待，祈祷没人占用GPU")
        # 1. 定义4位量化配置（替代原来的load_in_4bit=True）
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,  # 启用4位量化
            bnb_4bit_use_double_quant=True,  # 双量化（进一步压缩）
            bnb_4bit_quant_type="nf4",  # 用NF4量化（QLORA推荐）
            bnb_4bit_compute_dtype=torch.float16  # 计算时用16位精度
        )
        model = AutoModelForCausalLM.from_pretrained(
            qwen_model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,  # 16位精度（核心修改1：去掉load_in_4bit）
            # load_in_4bit=True,  # QLORA的量化配置，LoRA需删除
            device_map="auto"
        ).eval()
        print('Qwen-7B-Chat 模型加载成功')
        return model

    except Exception as e:
        error_msg = str(e)
        if "out of memory" in error_msg.lower():
            print(f"❌ 显存不足！解决方案：1. 改用 CPU 加载（把 device_map 设为 'cpu'）；2. 用 Int4 量化模型（需装 auto-gptq）")
        elif "No such file or directory" in error_msg:
            print(f"❌ 模型路径错误！请确认 {model_path} 下有 config.json、pytorch_model-*.bin 等文件")
        else:
            print(f"❌ 模型加载失败：{error_msg}")
        return None


In [5]:
if qwen_tokenizer:
    qwen_model = load_qwen_model(qwen_model_path)
else:
    print('分词器未加载，无法加载模型')

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


开始加载模型，模型较大，耐心等待，祈祷没人占用GPU


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Qwen-7B-Chat 模型加载成功


In [6]:

# 3. 配置QLORA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # QWen注意力层
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
# 装LoRA适配器
qwen_model = get_peft_model(qwen_model, lora_config)
qwen_model.print_trainable_parameters()  # 仍输出~0.01%可训练参数

trainable params: 4,194,304 || all params: 7,725,518,848 || trainable%: 0.0543


In [7]:
# 检查模型所有参数的 requires_grad 状态
print("模型参数状态（前10个参数示例）：")
for name, param in list(qwen_model.named_parameters())[:10]:
    print(f"参数名：{name:50} | 是否可训练：{param.requires_grad} | 形状：{param.shape}")

# 统计可训练参数数量
trainable_params = [p for p in qwen_model.parameters() if p.requires_grad]
print(f"\n可训练参数总数：{len(trainable_params)}")
print(f"可训练参数占比：{len(trainable_params)/len(list(qwen_model.parameters()))*100:.6f}%")
# 检查QLORA适配器是否生效
qwen_model.print_trainable_parameters()  # 应输出 ~0.01% 的可训练参数

# 检查模型是否为 PeftModel 类型（确保适配器已安装）
print(f"模型类型：{type(qwen_model)}")  # 应输出 <class 'peft.peft_model.PeftModelForCausalLM'>

模型参数状态（前10个参数示例）：
参数名：base_model.model.transformer.wte.weight            | 是否可训练：False | 形状：torch.Size([151936, 4096])
参数名：base_model.model.transformer.h.0.ln_1.weight       | 是否可训练：False | 形状：torch.Size([4096])
参数名：base_model.model.transformer.h.0.attn.c_attn.base_layer.weight | 是否可训练：False | 形状：torch.Size([12288, 4096])
参数名：base_model.model.transformer.h.0.attn.c_attn.base_layer.bias | 是否可训练：False | 形状：torch.Size([12288])
参数名：base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight | 是否可训练：True | 形状：torch.Size([8, 4096])
参数名：base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight | 是否可训练：True | 形状：torch.Size([12288, 8])
参数名：base_model.model.transformer.h.0.attn.c_proj.weight | 是否可训练：False | 形状：torch.Size([4096, 4096])
参数名：base_model.model.transformer.h.0.ln_2.weight       | 是否可训练：False | 形状：torch.Size([4096])
参数名：base_model.model.transformer.h.0.mlp.w1.weight     | 是否可训练：False | 形状：torch.Size([11008, 4096])
参数名：base_model.model.transformer.h.0.mlp.w2.weight     |

In [8]:
from torch.optim import AdamW

# 仅获取可训练参数（QLORA的低秩矩阵参数）
trainable_params = [p for p in qwen_model.parameters() if p.requires_grad]

# 定义原生 AdamW 优化器（不经过 accelerate 封装）
optimizer = AdamW(
    trainable_params,
    lr=2e-4,  # 学习率和之前保持一致
    weight_decay=0.01  # 可选：添加权重衰减防止过拟合
)

In [9]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer  # 关键：导入SFTTrainer

# --------------------------------------------------------------------------
json_data_path = '/home/wby/projects/week5/data/food_task.json'
output_dir = '/home/wby/projects/week5/data/qwen_food_lora' #存放QLORA适配器的地方

# -------------------------- 新增：猴子补丁（关键修复） --------------------------
from torch.optim import AdamW
from trl import SFTTrainer  # 关键：导入SFTTrainer

# 给 AdamW 动态添加一个空的 train() 方法，避免 AttributeError
def dummy_train(self):
    pass  # 什么都不做，仅为了避免报错

# 把空方法绑定到 AdamW 类上
if not hasattr(AdamW, "train"):
    AdamW.train = dummy_train
# --------------------------------------------------------------------------------


# 4. 加载并格式化数据集（SFTTrainer需指定“文本列”，简化格式）
def format_dataset(example):
    # 拼接“指令+输入+输出”为单文本（SFTTrainer默认读取“text”列）
    example["text"] = f"""### 指令：{example['instruction']}
### 输入：{example['input']}
### 输出：{example['output']}"""
    return example

dataset = load_dataset("json", data_files=json_data_path)["train"]
formatted_dataset = dataset.map(format_dataset, remove_columns=dataset.column_names)

# 5. 配置训练参数（新增优化器指定，关键修复）
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=20,
    logging_steps=5,
    save_strategy="epoch",
    # fp16=True,  # QLORA需开启混合精度，LoRA用16位模型可删除（也可保留）
    optim="adamw_torch"  # 直接用原生AdamW，不会再报train()错
)

# 用SFTTrainer（推荐，流程简单）
trainer = SFTTrainer(
    model=qwen_model,
    args=training_args,
    train_dataset=formatted_dataset,
    tokenizer=qwen_tokenizer,
    max_seq_length=128,
    peft_config=lora_config,
    dataset_text_field="text"
)
trainer.train()  # 此时不会再报 AttributeError！

# 8. 保存适配器（和之前一致）
qwen_model.save_pretrained(f"{output_dir}/final_lora")
print(f"微调完成！适配器保存在：{output_dir}/final_lora")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/training_args.py:2063: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:402: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


Step,Training Loss
5,2.285900
10,0.837500
15,0.468800
20,0.280300
25,0.163200
30,0.147100
35,0.135300
40,0.129400
45,0.132500
50,0.114100


微调完成！适配器保存在：/home/wby/projects/week5/data/qwen_food_lora/final_lora
